In [ ]:
import pandas as pd
import numpy as np

# Loading the data
df = pd.read_csv('final_pose_features_dataset.csv')

print(df.shape)
print(df.columns)
print(df.head())

(14138, 193)
Index(['x0_scaled', 'y0_scaled', 'z0_scaled', 'x1_scaled', 'y1_scaled',
       'z1_scaled', 'x2_scaled', 'y2_scaled', 'z2_scaled', 'x3_scaled',
       ...
       'acc_norm_24', 'acc_norm_25', 'acc_norm_26', 'acc_norm_27',
       'acc_norm_28', 'acc_norm_29', 'acc_norm_30', 'acc_norm_31',
       'acc_norm_32', 'acc_mean_norm'],
      dtype='object', length=193)
   x0_scaled  y0_scaled  z0_scaled  x1_scaled  y1_scaled  z1_scaled  \
0  -0.290230  -0.908041  -1.103772  -0.272303  -0.967773  -1.157645   
1  -0.299645  -0.876496  -1.323764  -0.282579  -0.938997  -1.383577   
2  -0.284160  -0.873014  -1.432610  -0.266014  -0.930987  -1.481315   
3  -0.246819  -0.866061  -1.383165  -0.227279  -0.922747  -1.444332   
4  -0.272222  -0.927351  -1.527996  -0.250786  -0.984679  -1.585891   

   x2_scaled  y2_scaled  z2_scaled  x3_scaled  ...  acc_norm_24  acc_norm_25  \
0  -0.261884  -0.966418  -1.157390  -0.250042  ...     0.000000     0.000000   
1  -0.272361  -0.938779  -1.383261  -

In [ ]:
# Identify feature columns (excluding ID and label)
feature_cols = [col for col in df.columns if col not in ['video_name', 'frame_id', 'label_x', 'label_y']]

# Inspect feature_cols
print("Feature Columns:", feature_cols)

Feature Columns: ['x0_scaled', 'y0_scaled', 'z0_scaled', 'x1_scaled', 'y1_scaled', 'z1_scaled', 'x2_scaled', 'y2_scaled', 'z2_scaled', 'x3_scaled', 'y3_scaled', 'z3_scaled', 'x4_scaled', 'y4_scaled', 'z4_scaled', 'x5_scaled', 'y5_scaled', 'z5_scaled', 'x6_scaled', 'y6_scaled', 'z6_scaled', 'x7_scaled', 'y7_scaled', 'z7_scaled', 'x8_scaled', 'y8_scaled', 'z8_scaled', 'x9_scaled', 'y9_scaled', 'z9_scaled', 'x10_scaled', 'y10_scaled', 'z10_scaled', 'x11_scaled', 'y11_scaled', 'z11_scaled', 'x12_scaled', 'y12_scaled', 'z12_scaled', 'x13_scaled', 'y13_scaled', 'z13_scaled', 'x14_scaled', 'y14_scaled', 'z14_scaled', 'x15_scaled', 'y15_scaled', 'z15_scaled', 'x16_scaled', 'y16_scaled', 'z16_scaled', 'x17_scaled', 'y17_scaled', 'z17_scaled', 'x18_scaled', 'y18_scaled', 'z18_scaled', 'x19_scaled', 'y19_scaled', 'z19_scaled', 'x20_scaled', 'y20_scaled', 'z20_scaled', 'x21_scaled', 'y21_scaled', 'z21_scaled', 'x22_scaled', 'y22_scaled', 'z22_scaled', 'x23_scaled', 'y23_scaled', 'z23_scaled', 'x24

In [ ]:
# Ensure proper sorting
df = df.sort_values(by=['video_name', 'frame_id'])

# Group by video and stack sequences
X = []
y = []

for video_id, group in df.groupby('video_name'):
    group = group.sort_values('frame_id')
    features = group[feature_cols].values
    label = group['label_x'].iloc[0]  # Assuming label per video

    X.append(features)
    y.append(label)

In [ ]:
# Padding sequences to same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = pad_sequences(X, padding='post', dtype='float32')  # shape -> (num_videos, max_frames, num_features)
y = np.array(y)

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (994, 34, 189)
y shape: (994,)


In [ ]:
# Checking the labels
print(y[:5])

['Bench_Press' 'Bench_Press' 'Bench_Press' 'Bench_Press' 'Bench_Press']


In [ ]:
# Encoding the labels
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)

num_classes = len(le.classes_)
print("Number of classes:", num_classes)

Number of classes: 8


In [ ]:
# Installing TCN Module for the neural network
!pip install keras-tcn

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, Masking
from tcn import TCN

# Define the sequential model
model = Sequential([
    # Input layer defining the expected input shape: (max_frames, num_features)
    Input(shape=(X.shape[1], X.shape[2])),
    # Masking layer to handle padded sequences (values equal to 0.0 will be masked)
    Masking(mask_value=0.0),
    # Temporal Convolutional Network layer
    TCN(nb_filters=64, kernel_size=5, return_sequences=False),
    # Dropout layer to prevent overfitting
    Dropout(0.3),
    # Dense layer with ReLU activation
    Dense(128, activation='relu'),
    # Another dropout layer
    Dropout(0.2),
    # Output layer with softmax activation for multi-class classification
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'tcn_2' (of type TCN) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ masking_2 (Masking)                  │ (None, 34, 189)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ tcn_2 (TCN)                          │ (None, 64)                  │         298,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 8)                   │           1,032 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 308,040 (1.18 MB)

 Trainable params: 308,040 (1.18 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
from sklearn.model_selection import train_test_split

# Split data: 80% train, 20% validation (stratified)
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_val, y_val))

Epoch 1/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.3042 - loss: 1.6500 - val_accuracy: 0.3317 - val_loss: 1.7058
Epoch 2/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.3800 - loss: 1.5557 - val_accuracy: 0.3116 - val_loss: 1.7818
Epoch 3/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.4036 - loss: 1.5677 - val_accuracy: 0.2563 - val_loss: 1.8582
Epoch 4/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.3943 - loss: 1.5159 - val_accuracy: 0.3618 - val_loss: 1.6550
Epoch 5/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.4066 - loss: 1.4742 - val_accuracy: 0.3568 - val_loss: 1.6541
Epoch 6/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.4306 - loss: 1.4405 - val_accuracy: 0.3869 - val_loss: 1.7114
Epoch 7/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.4667 - loss: 1.3914 - val_accuracy: 0.3467 - val_loss: 1.5306
Epoch 8/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.4833 - loss: 1.3236 - val_accu

In [33]:
# Evaluate on validation/test set
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4268 - loss: 2.6957
Validation Accuracy: 0.4472


In [34]:
# Save model in Keras format
model.save('tcn_pose_model.keras')

In [35]:
# Saving the model in the Cloud Storage Bucket
!gsutil cp tcn_pose_model.keras gs://exercise-recognition-dataset/model/

Copying file://tcn_pose_model.keras [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/3.7 MiB.                                      


In [ ]:
import joblib

joblib.dump(le, 'label_encoder.joblib')

['label_encoder.joblib']

In [ ]:
!gsutil cp label_encoder.joblib gs://exercise-recognition-dataset/model/

Copying file://label_encoder.joblib [Content-Type=application/octet-stream]...
/ [1 files][  871.0 B/  871.0 B]                                                
Operation completed over 1 objects/871.0 B.                                      
